In [1]:
import pandas as pd

In [2]:
# Load the CSV files
food_portion_df = pd.read_csv('../FoodData_Central_csv_2024-04-18/FoodData_Central_csv_2024-04-18/food_portion.csv')
food_df = pd.read_csv('../FoodData_Central_csv_2024-04-18/FoodData_Central_csv_2024-04-18/food.csv')

In [3]:
# Filter food_portion_df to include only rows where modifier is not null
food_portion_filtered = food_portion_df[food_portion_df['modifier'].notna()]

# Perform a left join with food_df
result_df = pd.merge(food_portion_filtered, food_df[['fdc_id', 'description']], 
                     on='fdc_id', how='left')

for index, row in result_df.iterrows():
    try:
        modifier_int = int(row['modifier'])
        result_df.at[index, 'modifier'] =  result_df.at[index, 'portion_description']
    except ValueError:
        continue

# Select only the columns we need
final_df = result_df[['fdc_id', 'description', 'amount', 'modifier', 'gram_weight']]

# Rename columns, sort, and save to csv
final_df = final_df.rename(columns={
    'description': 'food_description',
    'amount': 'quantity',
    'modifier': 'natural_unit',
    'gram_weight': 'grams'
})

final_df = final_df.sort_values('fdc_id')
final_df = final_df[~final_df['natural_unit'].str.contains("Quantity not specified", case=False, na=False)]
final_df = final_df.reset_index(drop=True)
final_df.to_csv('food_unit_conversion.csv', index=False)

In [4]:
print("Conversion table has been created and saved as 'food_unit_conversion.csv'")
print("\nFirst few rows of the conversion table:")
print(final_df.head())

Conversion table has been created and saved as 'food_unit_conversion.csv'

First few rows of the conversion table:
   fdc_id                                   food_description  quantity  \
0  167512  Pillsbury Golden Layer Buttermilk Biscuits, Ar...       1.0   
1  167513  Pillsbury, Cinnamon Rolls with Icing, refriger...       1.0   
2  167514  Kraft Foods, Shake N Bake Original Recipe, Coa...       1.0   
3  167515     George Weston Bakeries, Thomas English Muffins       1.0   
4  167516         Waffles, buttermilk, frozen, ready-to-heat       1.0   

                natural_unit  grams  
0                    serving   34.0  
1  serving 1 roll with icing   44.0  
2                    serving   28.0  
3                    serving   57.0  
4             waffle, square   39.0  
